# PASO 5: LECTURA DE DATOS DE UN CSV

In [19]:
# DATOS

import pandas as pd

df = pd.read_csv('P_C04-1C2025_data.csv', sep = ';')

nec = {}

for _, fila in df.iterrows():
    nec.update({(fila.LOT, fila.PRODUCT): fila.QTY} )


In [20]:
# MODELO

prod = {
    'Y4W': 
    {
    'corte_m1': 0,
    'corte_m2': 0,
    'doblado': 0,
    'soldado': 0,
    'corte_w': 23,
    'tapizado': 0,
    'pintura': 15,
    'despacho': 24
    },
    'Q79': 
    {
    'corte_m1': 45,
    'corte_m2': 40,
    'doblado': 23,
    'soldado': 32,
    'corte_w': 34,
    'tapizado': 102,
    'pintura': 36,
    'despacho': 34
    },
    'VEQ': 
    {
    'corte_m1': 48,
    'corte_m2': 43,
    'doblado': 38,
    'soldado': 56,
    'corte_w': 0,
    'tapizado': 0,
    'pintura': 36,
    'despacho': 20
    }
}

op = ['corte_m1','corte_m2','doblado','soldado','corte_w','tapizado','pintura','despacho']

ultima_op = 'despacho'

from docplex.cp.model import CpoModel

m = CpoModel()

ope = []

# intervalos
for i in nec:
    r = []
    for j in op:      
        r.append(m.interval_var(optional = True, size = prod[i[1]][j]*nec[i], name = i[0] + '_' + i[1] + '_' +  j ))
    ope.append(r)

# relaciones de precedencia
for i in nec:
    idx = list(nec.keys()).index(i) # consulta de stackoverflow
    if i[1] == 'Y4W':   # estanteria de madera
        m.add(m.presence_of(ope[idx][op.index('corte_m1')])==0)
        m.add(m.presence_of(ope[idx][op.index('corte_m2')])==0)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==0)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==0)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('pintura')]))
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))

    if i[1] == 'Q79':   # silla combinada
        m.add(m.presence_of(ope[idx][op.index('corte_m1')]) + m.presence_of(ope[idx][op.index('corte_m2')]) ==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==1)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)        
        m.add(m.end_before_start(ope[idx][op.index('corte_m1')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('corte_m2')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('tapizado')]))
        m.add(m.end_before_start(ope[idx][op.index('tapizado')], ope[idx][op.index('despacho')]))
    if i[1] == 'VEQ':   # silla de metal  
        m.add(m.presence_of(ope[idx][op.index('corte_m1')]) + m.presence_of(ope[idx][op.index('corte_m2')]) ==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==0)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_m1')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('corte_m2')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))

# agrupacion (span)
lot = []
for i in nec:
    lot.append(m.interval_var(name = i[0]))

for l in range(len(lot)):
    m.add(m.span(lot[l], [ope[l][j] for j in range(len(op))]))


# funcional
m.add( m.minimize(m.max(m.end_of(ope[i][op.index(ultima_op)]) for i in range(len(nec)))))

# resolución
s = m.solve()

s.print_solution()



 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 126 variables, 186 constraints
 ! Presolve      : 9 extractables eliminated
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 529.4 (before), 529.4 (after)
 !  . Memory usage      : 511.3 kB (before), 511.3 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        126                 -
 + New bound is 25330
 ! Using iterative diving.
 *         26770      170  0.03s        1      (gap is 5.38%)
 *         26025      357  0.03s        1      (gap is 2.67%)
           26025      358         82    1   F        -
 + New bound is 26023 (gap is 0.01%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 2 solutions found.
 ! B

In [ ]:
print(m.export_as_cpo())